In [1]:
# !pip install --upgrade scikit-image
# !pip install tqdm
!pip install tensorflow==1.15.0
!pip install keras==2.0.0


In [1]:
!git clone https://github.com/Tony607/efficientnet_keras_transfer_learning
%cd efficientnet_keras_transfer_learning/

In [1]:
import cv2
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import json
import os
from tqdm import tqdm, tqdm_notebook

from tensorflow.keras.layers import GlobalAveragePooling2D,Input, Conv2D, MaxPooling2D, Flatten, Dense, Dropout, BatchNormalization,AveragePooling2D
from tensorflow.keras import models
from tensorflow.keras import layers
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import optimizers
from tensorflow.keras.applications.resnet50 import ResNet50,preprocess_input,ResNet50

from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras.callbacks import EarlyStopping

from sklearn.metrics import confusion_matrix,classification_report
from efficientnet import EfficientNetB0 as Net
from efficientnet import center_crop_and_resize, preprocess_input

In [1]:
train_dir='../../input/blood-cells/dataset2-master/dataset2-master/images/TRAIN'
validation_dir='../../input/blood-cells/dataset2-master/dataset2-master/images/TEST'
height=200
width=200
batch_size=1
epochs=10
NUM_TRAIN=50
NUM_TEST=50
input_shape=(200,200,3)
train_datagen = ImageDataGenerator(preprocessing_function=preprocess_input
                                   
        
)

# Note that the validation data should not be augmented!
test_datagen = ImageDataGenerator()

# define the ImageNet mean subtraction (in RGB order) and set the
# the mean subtraction value for each of the data augmentation
# objects
mean = np.array([123.68, 116.779, 103.939], dtype="float32")
train_datagen.mean = mean
test_datagen.mean = mean

train_generator = train_datagen.flow_from_directory(
    # This is the target directory
    train_dir,
    # All images will be resized to target height and width.
    target_size=(height, width),
    batch_size=batch_size,
    color_mode="rgb",
    # Since we use categorical_crossentropy loss, we need categorical labels
    class_mode="categorical",
)

validation_generator = test_datagen.flow_from_directory(
    validation_dir,
    target_size=(height, width),
    batch_size=batch_size,
    color_mode="rgb",
    class_mode="categorical",
)




### Using EfficientNet EfficientNetB0

In [1]:
from tensorflow.keras.constraints import max_norm
# loading pretrained conv base model
conv_base = Net(weights="imagenet", include_top=False, input_shape=input_shape)
headModel = conv_base.output
headModel = GlobalAveragePooling2D()(headModel)
headModel = Flatten(name="flatten")(headModel)
headModel = BatchNormalization()(headModel)
headModel = Dense(256, activation="relu", kernel_constraint=max_norm(3), bias_constraint=max_norm(3))(headModel)
headModel = Dropout(0.5)(headModel)
headModel = Dense(4, activation="sigmoid", kernel_constraint=max_norm(3), bias_constraint=max_norm(3))(headModel)
# place the head FC model on top of the base model (this will become
# the actual model we will train)
model = models.Model(inputs=conv_base.input, outputs=headModel)
# loop over all layers in the base model and freeze them so they will
# *not* be updated during the training process
for layer in conv_base.layers:
    layer.trainable = False
# compile the model
opt = optimizers.SGD(lr=1e-4, momentum=0.9)
model.compile(loss="categorical_crossentropy", optimizer=opt,metrics=["accuracy"])


In [1]:

history = model.fit_generator(
    train_generator,
    steps_per_epoch=NUM_TRAIN // batch_size,
    epochs=epochs,
    validation_data=validation_generator,
    validation_steps=NUM_TEST // batch_size,
    verbose=1
)

In [1]:
predictions = model.predict_generator(
    validation_generator,
    steps=validation_generator.n / validation_generator.batch_size,
    verbose=1)

In [1]:
y_pred = np.argmax(predictions, axis=1)
print('Confusion Matrix')
print(confusion_matrix(validation_generator.classes, y_pred))
print('Classification Report')
target_names = ['altogrado', 'ascus', 'bajogrado','benigna']
print(classification_report(validation_generator.classes, y_pred, target_names=target_names))

## ResNet 

In [1]:
baseModel = ResNet50(weights="imagenet", include_top=False,input_tensor=Input(shape=input_shape))

In [1]:
from tensorflow.keras.constraints import max_norm
headModel = baseModel.output
headModel = Flatten(name="flatten")(headModel)
headModel = Dropout(0.5)(headModel)
headModel = BatchNormalization()(headModel)
headModel = Dense(4, activation="sigmoid", kernel_constraint=max_norm(3), bias_constraint=max_norm(3))(headModel)
# place the head FC model on top of the base model (this will become
# the actual model we will train)
model = models.Model(inputs=baseModel.input, outputs=headModel)
# loop over all layers in the base model and freeze them so they will
# *not* be updated during the training process

for layer in baseModel.layers:
    layer.trainable = False
    
    if layer.name.startswith('bn'):
        layer.call(layer.input, training=False)
# compile the model
opt = optimizers.SGD(lr=1e-4, momentum=0.9)
model.compile(loss="categorical_crossentropy", optimizer=opt,metrics=["accuracy"])   
    

In [1]:
es_callback = EarlyStopping(monitor='val_loss', patience=5)
history = model.fit_generator( train_generator,
                            steps_per_epoch=NUM_TRAIN // batch_size,
                            epochs=epochs,
                            validation_data=validation_generator,
                            validation_steps=NUM_TEST // batch_size,
                            verbose=1,callbacks=[es_callback]
                        )

In [1]:
predictions = model.predict_generator(
    validation_generator,
    steps=validation_generator.n / validation_generator.batch_size,
    verbose=1)

In [1]:
y_pred = np.argmax(predictions, axis=1)
print('Confusion Matrix')
print(confusion_matrix(validation_generator.classes, y_pred))
print('Classification Report')
target_names = ['altogrado', 'ascus', 'bajogrado','benigna']
print(classification_report(validation_generator.classes, y_pred, target_names=target_names))

### vgg16 binary classifier

In [1]:
base_model = VGG16(weights='imagenet', include_top=False, input_shape=(200, 200, 3))
add_model = models.Sequential()
add_model.add(Flatten(input_shape=base_model.output_shape[1:]))
add_model.add(Dense(256, activation='relu'))
add_model.add(Dense(1, activation='sigmoid'))

model = models.Model(inputs=base_model.input, outputs=add_model(base_model.output))
model.compile(loss='binary_crossentropy', optimizer=optimizers.SGD(lr=1e-4, momentum=0.9),
              metrics=['accuracy'])

model.summary()

In [1]:
validation_generator.classes=np.where(validation_generator.classes==3,1,0)
train_generator.classes=np.where(train_generator.classes==3,1,0)
validation_generator.class_mode='binary'
train_generator.class_mode='binary'

In [1]:
train_generator.class_indices={'EOSINOPHIL': 0, 'LYMPHOCYTE': 0, 'MONOCYTE': 0, 'NEUTROPHIL': 1}
validation_generator.class_indices={'EOSINOPHIL': 0, 'LYMPHOCYTE': 0, 'MONOCYTE': 0, 'NEUTROPHIL': 1}

In [1]:
train_generator.num_classes=2
validation_generator.num_classes=2


In [1]:
history = model.fit_generator( train_generator,
                            steps_per_epoch=NUM_TRAIN // batch_size,
                            epochs=1,
                            validation_data=validation_generator,
                            validation_steps=NUM_TEST // batch_size,
                            verbose=1,
                            use_multiprocessing=True,
                            workers=4,
                        )

In [1]:
predictions = model.predict_generator(
    validation_generator,
    steps=validation_generator.n / validation_generator.batch_size,
    verbose=1)

In [1]:
y_pred = np.argmax(predictions, axis=1)
print('Confusion Matrix')
print(confusion_matrix(validation_generator.classes, y_pred))
print('Classification Report')
target_names = ['altogrado', 'ascus', 'bajogrado','benigna']
print(classification_report(validation_generator.classes, y_pred, target_names=target_names))